In [47]:
import numpy as np
from IPython.display import Markdown, display

# 目标层的成对比较矩阵 A
A = np.array([[1, 1/2, 4, 3, 3], 
              [2, 1, 7, 5, 5],
              [1/4, 1/7, 1, 1/2, 1/3],
              [1/3, 1/5, 2, 1, 1],
              [1/3, 1/5, 3, 1, 1]])

# 准则层的成对比较矩阵 B1, B2, B3, B4, B5
B = [
    np.array([[1, 2, 5], [1/2, 1, 2], [1/5, 1/2, 1]]),
    np.array([[1, 1/3, 1/8], [3, 1, 1/3], [8, 3, 1]]),
    np.array([[1, 1, 3], [1, 1, 3], [1/3, 1/3, 1]]),
    np.array([[1, 3, 4], [1/3, 1, 1], [1/4, 1, 1]]),
    np.array([[1, 1, 1/4], [1, 1, 1/4], [4, 4, 1]])
]

# 定义一个函数,用于计算成对比较矩阵的最大特征值和对应的特征向量
def eigen_value_vector(matrix):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    max_idx = np.argmax(eigenvalues)
    max_eigenvalue = eigenvalues[max_idx].real
    max_eigenvector = eigenvectors[:, max_idx].real
    return max_eigenvalue, max_eigenvector


In [48]:
display(Markdown("## 步骤1:计算各个成对比较矩阵的特征向量"))

# 计算目标层成对比较矩阵的最大特征值和对应的特征向量
max_eigenvalue_A, max_eigenvector_A = eigen_value_vector(A)

# 计算准则层成对比较矩阵的最大特征值和对应的特征向量
max_eigenvector_list = []
for i, matrix in enumerate(B, start=1):
    max_eigenvalue, max_eigenvector = eigen_value_vector(matrix)
    max_eigenvector_list.append(max_eigenvector)

# 使用 Pandas DataFrame 显示最大特征向量结果
max_eigenvector_df = pd.DataFrame({
    '最大特征向量': [np.round(max_eigenvector_A, 4).tolist()] + [np.round(max_eigenvector, 4).tolist() for max_eigenvector in max_eigenvector_list]
}, index=['目标层矩阵 A'] + [f'准则{i}矩阵' for i in range(1, len(B)+1)])

display(Markdown(max_eigenvector_df.to_markdown()))

## 步骤1:计算各个成对比较矩阵的特征向量

|              | 最大特征向量                                 |
|:-------------|:---------------------------------------------|
| 目标层矩阵 A | [-0.4658, -0.8409, -0.0951, -0.1733, -0.192] |
| 准则1矩阵    | [-0.8902, -0.4132, -0.1918]                  |
| 准则2矩阵    | [-0.1128, -0.3255, -0.9388]                  |
| 准则3矩阵    | [0.6882, 0.6882, 0.2294]                     |
| 准则4矩阵    | [0.9255, 0.2803, 0.2547]                     |
| 准则5矩阵    | [0.2357, 0.2357, 0.9428]                     |

In [49]:
display(Markdown("## 步骤2:进行一致性检验"))

display(Markdown("一致性指标 $CI$ 的计算公式:"))
display(Markdown("$CI = \\frac{\\lambda_{\\max} - n}{n - 1}$"))

display(Markdown("一致性比率 $CR$ 的计算公式:"))
display(Markdown("$CR = \\frac{CI}{RI}$"))

display(Markdown("当 $CR < 0.1$ 时,判断通过一致性检验。"))

# Saaty给出的随机一致性指标 RI
RI = [0, 0, 0.58, 0.90, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49]

# 检验目标层矩阵 A 的一致性
n = A.shape[0]
CI_A = (max_eigenvalue_A - n) / (n - 1)
CR_A = CI_A / RI[n-1]

# 检验准则层矩阵 B1, B2, B3, B4, B5 的一致性
CI_list = []
CR_list = []
consistency_list = []
for i, matrix in enumerate(B, start=1):
    max_eigenvalue, _ = eigen_value_vector(matrix)
    n = matrix.shape[0]
    CI = (max_eigenvalue - n) / (n - 1)
    CR = CI / RI[n-1]
    CI_list.append(CI)
    CR_list.append(CR)
    consistency_list.append('是' if CR < 0.1 else '否')

# 使用 Pandas DataFrame 显示一致性检验结果
consistency_df = pd.DataFrame({
    '一致性指标 CI': [np.round(CI_A, 4)] + [np.round(CI, 4) for CI in CI_list],
    '一致性比率 CR': [np.round(CR_A, 4)] + [np.round(CR, 4) for CR in CR_list],
    '是否通过一致性检验': ['是' if CR_A < 0.1 else '否'] + consistency_list
}, index=['目标层矩阵 A'] + [f'准则{i}矩阵' for i in range(1, len(B)+1)])

display(Markdown(consistency_df.to_markdown()))

## 步骤2:进行一致性检验

一致性指标 $CI$ 的计算公式:

$CI = \frac{\lambda_{\max} - n}{n - 1}$

一致性比率 $CR$ 的计算公式:

$CR = \frac{CI}{RI}$

当 $CR < 0.1$ 时,判断通过一致性检验。

|              |   一致性指标 CI |   一致性比率 CR | 是否通过一致性检验   |
|:-------------|----------------:|----------------:|:---------------------|
| 目标层矩阵 A |          0.018  |          0.0161 | 是                   |
| 准则1矩阵    |          0.0028 |          0.0048 | 是                   |
| 准则2矩阵    |          0.0008 |          0.0013 | 是                   |
| 准则3矩阵    |         -0      |         -0      | 是                   |
| 准则4矩阵    |          0.0046 |          0.0079 | 是                   |
| 准则5矩阵    |          0      |          0      | 是                   |

In [50]:
display(Markdown('## 步骤3:确定权向量'))

# 确定目标层矩阵 A 的权向量
W_A = max_eigenvector_A / np.sum(max_eigenvector_A)

# 确定准则层矩阵 B1, B2, B3, B4, B5 的权向量
W_list = []
for i, matrix in enumerate(B, start=1):
    _, max_eigenvector = eigen_value_vector(matrix)
    W = max_eigenvector / np.sum(max_eigenvector)
    W_list.append(W)

# 使用 Pandas DataFrame 显示权向量结果
weights_df = pd.DataFrame({
    '权向量': [np.round(W_A, 4).tolist()] + [np.round(W, 4).tolist() for W in W_list]
}, index=['目标层矩阵 A'] + [f'准则{i}矩阵' for i in range(1, len(B)+1)])

display(Markdown(weights_df.to_markdown()))

## 步骤3:确定权向量

|              | 权向量                                   |
|:-------------|:-----------------------------------------|
| 目标层矩阵 A | [0.2636, 0.4758, 0.0538, 0.0981, 0.1087] |
| 准则1矩阵    | [0.5954, 0.2764, 0.1283]                 |
| 准则2矩阵    | [0.0819, 0.2363, 0.6817]                 |
| 准则3矩阵    | [0.4286, 0.4286, 0.1429]                 |
| 准则4矩阵    | [0.6337, 0.1919, 0.1744]                 |
| 准则5矩阵    | [0.1667, 0.1667, 0.6667]                 |

In [51]:
display(Markdown('## 步骤4:计算方案对目标的综合权重'))

# 计算方案对目标的综合权重
W_B = np.array([W.tolist() for W in W_list])
W_P = W_B.T.dot(W_A)

display(Markdown('令 $\\boldsymbol{W}_B = (\\boldsymbol{w}_{B1}, \\boldsymbol{w}_{B2}, \\boldsymbol{w}_{B3}, \\boldsymbol{w}_{B4}, \\boldsymbol{w}_{B5})$,'))
display(Markdown('则方案对目标的综合权重为:'))
display(Markdown('$\\boldsymbol{W}_P = \\boldsymbol{W}_B^T \\cdot \\boldsymbol{w}_A$'))

# 使用 Pandas DataFrame 显示综合权重结果
weights_P_df = pd.DataFrame(W_P.reshape(1, -1), 
                            index=['综合权重'],
                            columns=[f'方案{i}' for i in range(1, len(W_P)+1)])

display(Markdown(weights_P_df.to_markdown()))

# 确定最终选择的目的地
best_plan = np.argmax(W_P) + 1
display(Markdown(f'综上所述,根据层次分析法,应该选择方案{best_plan}作为最终的目的地。'))

## 步骤4:计算方案对目标的综合权重

令 $\boldsymbol{W}_B = (\boldsymbol{w}_{B1}, \boldsymbol{w}_{B2}, \boldsymbol{w}_{B3}, \boldsymbol{w}_{B4}, \boldsymbol{w}_{B5})$,

则方案对目标的综合权重为:

$\boldsymbol{W}_P = \boldsymbol{W}_B^T \cdot \boldsymbol{w}_A$

|          |    方案1 |    方案2 |    方案3 |
|:---------|---------:|---------:|---------:|
| 综合权重 | 0.299255 | 0.245304 | 0.455441 |

综上所述,根据层次分析法,应该选择方案3作为最终的目的地。